In [1]:
import cv2
import matplotlib.pyplot as plt
import mediapipe as mp
import math

In [106]:
model_path = 'pose_landmarker_full.task'

BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE,
)

In [3]:
def angle(l1, l2):
    dx = l2.x - l1.x
    dy = l2.y - l1.y
    angle_rad = math.atan2(dy, dx)
    angle_deg = math.degrees(angle_rad)
    return angle_deg

def eval_horiz(angle, horiz_tol):
    angle = abs(angle)
    return angle <= horiz_tol or abs(angle - 180) <= horiz_tol

def compute_posture_heuristic(landmarks):
    left_shoulder = landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value]
    left_eye = landmarks[mp.solutions.pose.PoseLandmark.LEFT_EYE.value]
    right_eye = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_EYE.value]
    nose = landmarks[mp.solutions.pose.PoseLandmark.NOSE.value]

    # Calculations general
    shoulder_angle = angle(left_shoulder, right_shoulder)
    shoulder_width = abs(right_shoulder.x - left_shoulder.x)
    shoulder_midpoint_x = (left_shoulder.x + right_shoulder.x) / 2
    nose_offset = abs(nose.x - shoulder_midpoint_x)
    head_tilt = angle(left_eye, right_eye)

    # Neck
    shoulder_midpoint_y = (left_shoulder.y + right_shoulder.y) / 2
    dx_neck = nose.x - shoulder_midpoint_x
    dy_neck = nose.y - shoulder_midpoint_y
    neck_vector_magnitude = math.sqrt(dx_neck**2 + dy_neck**2)

    if neck_vector_magnitude > 0:
        cos_theta = (-dy_neck) / neck_vector_magnitude
        cos_theta = max(min(cos_theta, 1), -1)
        neck_tilt_angle = math.degrees(math.acos(cos_theta))
    else:
        neck_tilt_angle = 0

    print(head_tilt)
    print(shoulder_angle)
    print(nose_offset)
    print(neck_vector_magnitude, )
    
    # Score
    score = 100

    if not eval_horiz(shoulder_angle, 15):
        score -= 30
    if not eval_horiz(head_tilt, 15):        
        score -= 20
    if nose_offset > 0.2 * shoulder_width:
        score -= 30
    if neck_tilt_angle > 15:
        score -= 20
    
    return score

In [121]:
# Singular image heuristic
image_path = 'bad2.png'

with PoseLandmarker.create_from_options(options) as landmarker:
    mp_image = mp.Image.create_from_file(image_path)
    result = landmarker.detect(mp_image)

compute_posture_heuristic(result.pose_landmarks[0])

RuntimeError: Unable to open file at c:\Users\alexl\AppData\Local\Programs\Python\Python312\Lib\site-packages/c:\Users\alexl\Code\hackprinceton\ml\pose_landmarker_full.task, errno=22

In [4]:
# Initialize drawing and pose detection
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Open webcam
cap = cv2.VideoCapture(0)

with mp_pose.Pose(
        static_image_mode=False,
        model_complexity=1,
        enable_segmentation=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # Convert the image to RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Run MediaPipe Pose
        results = pose.process(image_rgb)

        # Draw pose landmarks on the frame
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS)
            
            # Compute the posture heuristic using the first landmark from the list
            heuristic = compute_posture_heuristic(results.pose_landmarks.landmark)
            
            # Overlay the computed posture heuristic on the frame
            cv2.putText(frame, 
                        f"Posture: {heuristic}", 
                        (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        1, 
                        (0, 255, 0), 
                        2, 
                        cv2.LINE_AA)

        # Display the frame
        cv2.imshow('MediaPipe Pose', frame)

        # Break loop on ESC key press
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()

-179.3807848636974
-173.893575178113
0.03448626399040222
0.322567023577303
-179.0869702561389
-174.25126909652298
0.03587782382965088
0.3211487840809111
-178.85060499341182
-174.51643567362572
0.03759057819843292
0.321236811411488
-178.79991704050255
-174.56649941239237
0.038570404052734375
0.321148616244906
-178.8734147216876
-174.6263254005023
0.03930562734603882
0.32058268433251425
-178.89697329685896
-174.6026245793
0.04018259048461914
0.3205668197829938
-178.75492978741883
-174.5897535046364
0.04226863384246826
0.32069338058154484
-178.6085742888438
-174.49051089996462
0.0432487428188324
0.32084402587054034
-178.47185484141028
-174.4372409482105
0.04419657588005066
0.3210019747933304
-178.33523464854113
-174.44801510350413
0.04539753496646881
0.3210474756754987
-178.2534402528213
-174.41482134998756
0.046701014041900635
0.3209468433339838
-178.19262512894895
-174.39450517042832
0.047608017921447754
0.3210633938839755
-178.15976688018964
-174.35762861214207
0.04848037660121918
0.32

In [2]:
import cv2
import mediapipe as mp

def process_image(image_path):
    # Initialize MediaPipe Pose in static image mode.
    mp_pose = mp.solutions.pose
    with mp_pose.Pose(
            static_image_mode=True,
            model_complexity=1,
            enable_segmentation=False,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as pose:

        # Load the image from the specified path.
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Image not found at path: {image_path}")

        # Convert the image to RGB.
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Process the image and obtain pose landmarks.
        results = pose.process(image_rgb)

        # Return the landmarks if detected, else None.
        return results.pose_landmarks.landmark if results.pose_landmarks else None

# Example usage:
landmarks = process_image("bad0.png")
if landmarks:
    print("Pose landmarks detected.")
else:
    print("No pose landmarks found.")


Pose landmarks detected.
